# Day Thirteen

## Task

The [task](https://adventofcode.com/2017/day/13) today is to determine the severity of a trip through a firewall. In this case, the firewall is made up of several layers. In each layer there is a scanner that moves up and down. The idea is that we have a preset path (in part one at least) for which we must compute the severity; the number of times that we encounter the scanner.

First, let's look in more detail at one of the layers. According to the task, a layer has a range and a depth. The range indicates how many steps the scanner has to travel across the layer. For example, the following layer has a range of three.

```
[ ]
[ ]
[ ]
```

The depth of a layer indicates where in the firewall the layer is. Here, we have a firewall with two layers, one with a range of three and depth of zero, and the other with a range of two and depth of one.

```
[ ]   [ ]
[ ]   [ ]
[ ]
```

Finally, let's look at how the scanner travels through a layer. It spends one tick in each step, so letting time increase from left to right, we have

```
t=0    t=1    t=2    t=3    t=4    t=5    t=6

[S]    [ ]    [ ]    [ ]    [S]    [ ]    [ ] 
[ ] -> [S] -> [ ] -> [S] -> [ ] -> [S] -> [ ] ...
[ ]    [ ]    [S]    [ ]    [ ]    [ ]    [S] 
```

### Part One

First, let's define a funtion that can give us the position of a scanner in a layer with given range after some time.

In [1]:
inc = lambda x: x + 1
dec = lambda x: x - 1

    
def position(range):
    p = -1
    f = inc

    while True:
        p = f(p)
        
        if p == 0:
            f = inc
        
        if p + 1 == range:
            f = dec

        yield p

Here, `position` returns a generator that on iteration gives the location of the scanner.

In [2]:
i = position(3)
for t in range(0, 5):
    print(f'At t={t}, scanner is about to leave position {next(i)}')

At t=0, scanner is about to leave position 0
At t=1, scanner is about to leave position 1
At t=2, scanner is about to leave position 2
At t=3, scanner is about to leave position 1
At t=4, scanner is about to leave position 0


Now let's think about the severity of a path through the firewall. According to the task, when `t` increments the following happens:

 - We move one step to the right.
 - If there is a scanner in the place we've just moved into, we're caught.
 - Then the scanners move one step in whatever direction they're travelling.
 
Using the example given in the task, we write this as follows:

In [3]:
def severity(firewall):
    sev = 0
    
    positions = [position(x) if x else None for x in firewall]
    
    for t in range(0, len(positions)):
        positions_at_t = [next(x) if x else None for x in positions]
        
        if positions_at_t[t] == 0:
            sev += t * firewall[t]
            
    return sev
            
            
severity([3, 2, None, None, 4, None, 4])

24

Using the puzzle data, this becomes

In [4]:
def read():
    with open('../../data/day13.txt') as f:
        data = f.read()
        
    data_dict = {}
    for x in data.split('\n'):
        if not x:
            continue
            
        k, v = map(int, x.split(': '))
        data_dict[k] = v
        
    return [data_dict.get(x, None) for x in range(0, max(data_dict.keys()))]


data = read()
severity(data)

1580

### Part Two

For part two, we are able to delay the start of our journey through the firewall. The idea is to find the smallest delay such that we don't get caught on our path. Note that this is not exactly the same as having a severity of zero (which is a necessary but not sufficient) since we might be caught in the first layer.

In code, I'm going to create a new function `caught` that looks very similar to `severity` but with the following changes:

 - To imitate the delay, I am going to tick my iterator before beginning.
 - So that being caught in the first layer isn't missed, I'm going to return a boolean value that indicates whether we've been caught.

In [5]:
def caught(firewall, delay=0):
    positions = [position(x) if x else None for x in firewall]
    
    for _ in range(0, delay):
        [next(x) if x else None for x in positions]
    
    for t in range(0, len(positions)):
        positions_at_t = [next(x) if x else None for x in positions]
        
        if positions_at_t[t] == 0:
            return True
            
    return False

data = read()

delay = 0
while True:
    if not caught(data, delay=delay):
        print(f'We sneak through with a delay of {delay} picoseconds!')
        break
    
    delay += 1
    
    if delay == 500:
        print(f'Bailing because this is taking a while...')
        break

Bailing because this is taking a while...


It turns out that the required delay is a bit bigger than I expected meaning I'll need to optimise my approach. The most obvious thing to do is to refine how I determine the start values of my scanners.

Notice that for a layer with a range of two, position over time takes the values `0, 1, 0, 1, 0, ...`. It has a cycle length of two. Similarly for a layer with a range of three, it has a cycle length of four. In general, a layer with range `r` has cycle length `2 * r - 2`. If we take `delay % cycle_length`, we find the number of times we need to iterate our starting positions.

In [6]:
def caught(firewall, delay=0):
    positions = [position(x) if x else None for x in firewall]
    
    for i, r in enumerate(firewall):
        if r:
            for _ in range(0, delay % (2 * r - 2)):
                next(positions[i])
    
    for t in range(0, len(positions)):
        positions_at_t = [next(x) if x else None for x in positions]
        
        if positions_at_t[t] == 0:
            return True
            
    return False

data = read()

delay = 0
while True:
    if not caught(data, delay=delay):
        print(f'We sneak through with a delay of {delay} picoseconds!')
        break
    
    delay += 1
    
    if delay % 50000 == 0:
        print(f'Delay: {delay}')

Delay: 50000
Delay: 100000
Delay: 150000
Delay: 200000
Delay: 250000
Delay: 300000
We sneak through with a delay of 339652 picoseconds!


In [7]:
data = [3, 2, None, None, 4, None, 4]

delay = 0
while True:
    if not caught(data, delay=delay):
        print(f'We sneak through with a delay of {delay} picoseconds!')
        break
    
    delay += 1
    
    if delay % 50000 == 0:
        print(f'Delay: {delay}')

We sneak through with a delay of 10 picoseconds!
